**Engy-4390** Nuclear Systems Design and Analysis, UMass Lowell, Chem. Eng. Nuclear Program Fall 2024

# Uranium Mill Network Dynamics Simulation Report
$
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

**Student(s):** Alyssa Puglisi <br>
**Course:** Engy-4390 Nuclear Systems Design and Analysis  <br>
**Mentor:** Prof. Valmor F. de Almeida

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**16Oct2024**

[Project Repository on `GitHub`](https://github.com/dpploy/engy-4390/tree/main/projects/2024/u-mill)

---
## Table of Contents<a id="toc"></a>
* [Problem Statment](#prob)
* [Introduction](#intro)
* [Objectives](#obj)
* [Tools](#tools)
* [Approach and Dynamics Equations](#app)
* [Challenges & Course Corrections](#correct)
* [Initialization of Solvent Extraction Module](#IPM)
* [Startup](#su)
* [Steady-State](#ss)
* [Disturbance](#malfunction)
* [References](#refs)
---

In [1]:
from cortix import ReactionMechanism

## [Problem Statement](#toc)<a id="prob"></a>

Design a dynamic system that accurately models the reactive mass transfer with multiple phases occuring in the solvent extraction process during White Mesa's uranium milling. 

Mechanism of Extraction & Stripping:

In [2]:
rxn_mech = ReactionMechanism(file_name='solvex-stripping-rxn-mechanism.txt', order_species=True)
rxn_mech.md_print()
#TODO C11H20O2 is only showing up as C1 H2 O2 ??

8 **Species:** 
 ${\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}} \quad  {\mathrm{H}\mathrm{S}\mathrm{O}_{4}^-}_\mathrm{(a)} \quad  {\mathrm{S}\mathrm{O}_{4}^-}_{\mathrm{(a)}} \quad  {\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}} \quad  {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)} \quad  {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}\bullet\mathrm{H}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}} \quad  {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}} \quad  {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}$

4 **Reactions:** 
 \begin{align*} 
{\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}}\ + \ 2.0\,{[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)}\ &\longleftrightarrow \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\\ 
{\mathrm{H}_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(a)}}\ + \ {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}}_\mathrm{(o)}\ &\longleftrightarrow \ {[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}\bullet\mathrm{H}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\\ 
{\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\ &\longleftrightarrow \ {\mathrm{S}\mathrm{O}_{4}^-}_{\mathrm{(a)}}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}\\ 
{\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}^-}_{\mathrm{(a)}}\ + \ 2.0\,{[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{S}\mathrm{O}_{4}}_{\mathrm{(o)}}\ &\longleftrightarrow \ 2.0\,{\mathrm{H}\mathrm{S}\mathrm{O}_{4}^-}_\mathrm{(a)}\ + \ {[[\mathrm{C}_{1}\mathrm{H}_{2}\mathrm{O}_{2}]_{3}\mathrm{N}\mathrm{H}]_{2}\mathrm{U}\mathrm{O}_{2}[\mathrm{S}\mathrm{O}_{4}]_{2}}_{\mathrm{(o)}}\\ 
\end{align*} 


The amine, on contact with a sulphuric acid solution, is converted into the sulphate or bisulphate salt.

The actual extraction reactions are complex. However, in a simplified way, the extraction can be depicted as an anion exchange where the anionic uranium complex from the aqueous solution replaces the sulphate or bisulphate ion in the organic phase. 

## [Introduction](#toc)<a id="intro"></a>


Solvent extraction, also known as liquid-liquid extraction, is the partition of metal complexes from two immiscible liquids. The two most common liquids used are on aqueous and one organic. Solvent extraction works on the exploitation of the relative solubility of the solute (metal complexes) in the two phases. The chemical potentials of each input make up the driving force of the transfer. Increased stabilitiy of the products as compared to the initial solution is necessary to increase the driving force. The solute enters the extraction system in the aqeuous feed stream and is extracted by the incoming solvent/organic stream. The feed is reduced and exits the extraction system as the raffinate stream; the solvent stream absorbs and exits the extraction system as the extract stream. Solvent extraction products do need further purification by methods such as percipitation, distillation, etc.

Solvent extraction is chosen as the necessary separation technique due to the high precision and accuracy of the separation. All based on the proper selection of the solvent, individual and specific complexes can be removed from the feed stream with substancial levels of exactness. Additional extraction tanks can be added to the system to see extraction percentages close to 100%. Morover, solvent extraction plus a distillation or percipitation process is chosen over just one of the former because of the increased simplicity of the distillation/precipitation process after solvent partitioning. All the benefits of solvent extraction are based on the choice of solvent.

The optimal choice solvent needs these three critical charactisterics:

* high affinitiy for the solute with low/no affinity for the carrier
* different in density when compared to the feed/carrier
* solute is easily recoverable from solvent

In an ideal world, other beneficial qualities of the choice solvent are:

* inexpensive
* non-toxic to humans and the environment
* non-volatile (low vapor pressure)
* non-flammable
* non-reactive
* non-corrosive

![Figure 1: Mixer Settler](symlink-pics/mixer-settler.jpg)

The uranium solvent extraction circuit consists of:

(a) Four extraction stages, 

(b) One organic washing stage,

(c) Four stripping stages, 

(d) One organic regeneration stage.

Solvent extraction occurs in mixer-settler vessels where the organic and aqueous phases are agitated in a mixer then deposited into a settling tank to phase split. The aqueous phase will sink to the bottom with the organic phase rising to the top. The organic phase is then sent to the previous stage with the aqueous phase being passed to the next stage. This results in the first mixer-settler vessel delivering the product to the stripping circuit with the final mixer-settler vessel delivering a waste stream to the tailings area.

After solvent extraction the stripping circuit begins with an acidic water wash. Immediately afterwards the organic phase is deposited into another bank of mixer-settler vessels where an acidic sodium chloride solution is used to extract uranyl ions. The stripping circuit will remove nearly all of the uranium from the organic phase resulting in a product of (NH<sub>4</sub>)<sub>2</sub>UO<sub>2</sub>(SO<sub>4</sub>)<sub>2</sub> which is ready to be fed into the precipitation stage where the sulfate will be removed from the compound to be precipitated into U<sub>3</sub>O<sub>8</sub>.


## [Objectives](#toc)<a id="obj"></a>

- Python code will be used to develop dynamic comprehensive systems that accurately represent the solvent extraction process. This is a non-equilibrium, reactive mass transfer scenario. The analysis involves both mathematical models of the system’s material streams and visual representations of the data on a time dependent basis.
- Deliverable content will be prepared using Jupyter notebooks to seamlessly integrate working code with descriptive markdown text.  
- A flowsheet based on White Mesa will be developed as the basis for our modules



## [Tools](#toc)<a id="tools"></a>

For this project we will be utilizing GitHub to store a remote repository of our python code and project report. GitHub is a version control software that programmers use to share and work on coding projects together. This allows for users to view the contribution history easily and rollback changes if needed. GitHub interfaces with git to create a graphical user interface for the git repository. This allows GitHub to add some of its own tools on top of git making it much more powerful. One of these tools is the Network Graph that allows users to see all branches of the repository and all commits along with dates that changes were made. Another tool is viewing the version history of a specific file by opening it on the GitHub repository. This shows all changes made to that file including commit numbers, dates of changes, and more. This makes it much easier to go back in time to see the history of the project and see who made which changes. At its core, a GitHub repository is a mirror of a git repository with any changes being submitted through GitHub automatically being sent to the git repository as well.

To work on a git repository, you first must clone the repository to your computer using the git clone command. This creates a clone of the remote repository on your computer which is then called the local repository. It will also create a copy of this local repository called the working directory. All edits made by the user are found in the working directory and do not impact the local repository until a commit is made. Next, the user will create their own branch using the git branch command. This branch is a copy of the repository that the user will use to make changes without impacting the main branch. This makes it easier for the repository maintainer to control the changes made to the project since they can view differences between the main branch and one of the user branches. This helps to prevent branch conflicts from creating bugs or other errors on the main branch of the repository. When a user is satisfied with a contribution, they will create a commit using the git commit command. The user will write a brief one-line description of what has been changed followed by a longer description with much more detail. A comparison is then made between the working directory and the local repository to see which changes have been made. Any changes in the working directory will be compiled into the commit and saved for later. Commits serve as documentation for changes and are vital to using git. The short description allows for the repository maintainer to easily see what changes are being made to the project, but still allows for technical details to be documented in the long description.

When a user wants to publish their commits to the remote repository for other users to see there is a specific procedure that must be followed. This is to prevent any conflicts from arising in the final step where the local repository is merged with the remote repository by the repository maintainer. First, the user must update their copy of the local repository to obtain the latest version of the project. Updating the local repository is done by using the git fetch command followed by the git merge command. This can also be done using the git pull command which executes both commands sequentially. The git fetch command compares the local repository to the target showing the differences between the two. This also downloads any missing commits to the local repository so they can be reviewed by the user. The git merge command combines the remote repository with the local repository. This process effectively creates a copy of the remote repository on the user's computer, but also includes any saved commits to ensure that the local repository is up to date. Following this process, the git push command can be used to push the unuploaded commits onto the remote repository. Since a separate branch was created earlier, this branch must be merged into the main branch so other users can merge these changes into their branch. This can be done by running git merge on the main branch to merge the main branch into the user's branch. Now the local repository is updated with the changes of all other users with the remaining commits that are trying to be published to the remote repository. Finally, the user can make a request to the repository maintainer to run git pull on their local repository. This is known as a pull request since you are requesting the repository maintainer to run the git pull command.

GitHub helps to make the pull request process easier by integrating it into their platform. All active pull requests can be viewed on the repository website with a discussion section where users can discuss any problems they find with the changes. In addition to this, GitHub streamlines the process through the use of the GitHub desktop app. Instead of running each of the git commands through the command line sequentially, they can be executed in a graphical user interface via GitHub desktop. After downloading GitHub desktop, the user is prompted to clone a repository and login to their GitHub account. This creates a local repository and working directory for the user with commits and pull requests being linked to their GitHub account. After creating changes to files in the working directory, GitHub desktop will show the user the current differences between the working directory and the local repository. A commit window is displayed with boxes for the short description and long description right next to these differences making the commit process much easier. In order to publish changes to the remote repository there are recommended prompts that aid the user through the pull, push, and pull request process. If the user would like to run any of the other git commands, these can be accessed at the top by clicking on Repository and selecting the desired command to run.

Cortix is a library for Python that was created by Dr. Almeida and his research group to simplify the process of producing dynamic system models. The full open-source code can be found on Github, where anyone can download and use the library and its available unit operation modules. The Github repository also contains examples that use the code to simulate various chemical interactions while delivering time dependent data. 

Jupyter Notebook is also open-source, and it is a way to integrate ordinary text with multiple programming languages such as Python. Python is a widely used programming language that has engineering modeling applications. This is a popular language for dynamic modelling and has been in use for over thirty years. It is commonplace in many programs used in the nuclear industry, making it a great candidate for this project. 

## [Approach and Dynamics Equations](#toc)<a id="app"></a>

-  First, a series of mass balances and component balances are made based on the desired input of 1 ton of raw material.
- These balances then take into account the chemical kinetics and related rates required in the operation of White Mesa's chemical reactions and fluid flow operations
- Once baseline models of each individual process exist, they are then programmed into 5 separate but connected modules within Cortix
- Each process will be capable of interacting with the modules connected to it on either end, allowing them to share data such as the incoming and outgoing streams from one process to the next

|     **Module**        |      **Inputs**      |      **Outputs**       |     **Legend**     |
|:----------------------|:---------------------|:-----------------------|:-------------------|
| Solvent Extraction             |m<sub>aef</sub>, m<sub>oef</sub>, m<sub>voii</sub>, m<sub>oii</sub>, m<sub>vaii</sub>, m<sub>aii</sub>        | m<sub>ap</sub>, m<sub>op</sub>, m<sub>voio</sub>, m<sub>oio</sub>, m<sub>vaio</sub>, m<sub>aio</sub>            |m<sub>aef</sub> = mass flow rate of aqueous external feed, m<sub>oef</sub> = mass flow rate of organic external feed, m<sub>voii</sub> = mass flow rate of vapor organic inter-stage inflow, m<sub>oii</sub> = mass flow rate of organic inter-stage inflow, m<sub>vaii</sub> = mass flow rate of vapor aqueous inter-stage inflow, m<sub>aii</sub> = mass flow rate of aqueous inter-stage inflow, m<sub>ap</sub> = mass flow rate of aqeuous product,m<sub>op</sub> = mass flow rate of organic product, m<sub>voio</sub> = mass flow rate of vapor organic inter-stage outflow, m<sub>oio</sub> = mass flow rate of organic inter-stage outflow, m<sub>vaio</sub> = mass flow rate of vapor aqueous inter-stage outflow, m<sub>aio</sub> = mass flow rate of aqueous inter-stage outflow           |             |        |           |        |

## [Challenges & Course Correctons](#toc)<a id="correct"></a>

## [Initialization of Solvent Extraction](#toc)<a id="IPM"></a>

```python   
                                      |
                                      |  Extraction Feed (from Decantation-Filtration filtrate)
                                      |
                                      v
                              |----------------|
                              |                |
         Organic Feed ------->|                |------> Organic Product (to Scrubbing internal)
         (internal source)    |    Solvent     |
                              |   Extraction   |
         Raffinate <----------|                |<------ Scrub Raffinate (from Scrubbing internal)
         Stream (to CCD Bank) |                |
                              |----------------|
          Organic Product <---|                |<------ Organic Feed (Organic Product from Solv. Extr. internal)
          (to Strip internal) |                |
                              |   Scrubbing    |
           Scrub Feed ------->|                |-------> Scrub Raffinate (to Solvent Extraction internal)
           (internal source)  |                |
                              |----------------|
          Organic Feed ------>|                |-------> Organic Regeneration (to Solvent Extraction not done)
        (from Scrub internal) |   Stripping    |
                              |                |<------ Stripping Feed (internal source)
                              |________________|<------ Stripping Feed (from Precipitation not implemented)
                                      |
                                      |
                                      |
                                      v
                               Stripping Product (Precipitation feed)
```

## [References](#toc)<a id="refs"></a>

- [1] “35 No. 9 Uranium Extraction Technology - IAEA.” Uranium Extraction Technology, IAEA, 1993, https://www-pub.iaea.org/MTCD/Publications/PDF/trs359_web.pdf. 
- [2] "31 No. 3 Manual on Laboratory Testing for Uranium Ore Precessing - IAEA." Manual on Laboratory Testing for Uranium Ore Precessing, IAEA, 1990, https://www-pub.iaea.org/MTCD/Publications/PDF/trs313_web.pdf